In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np

import torch
import torchvision as tv

import noiseprofile as noise
import IntelSceneDataset as isds
import classifiers
from classifiers import SimpleClassifier

import denoise

from tqdm.auto import tqdm

from sporco import util
from sporco import linalg
from sporco import plot
plot.config_notebook_plotting()
from sporco.cupy import (cupy_enabled, np2cp, cp2np, select_device_by_load,
                         gpu_info)
from sporco.cupy.dictlrn import onlinecdl
from sporco.cupy.admm import cbpdn
from sporco.cupy import cnvrep
from sporco.cupy import linalg as cplinalg
from sporco.cupy import prox as cpprox
from sporco.cupy.linalg import irfftn,rfftn
from sporco.cupy.linalg import inner

import os

import matplotlib.pyplot as plt

In [2]:
transform = tv.transforms.Compose(
    [tv.transforms.CenterCrop(150),
     tv.transforms.ToTensor(),
     tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
dbpath = "/mnt/hd-storage/IntelImageClassification"

baseoutdir = "./data/out"



cleanTe = isds.IntelSceneDataset(dbpath, segment="test", transform=transform)

In [4]:
# clf = SimpleClassifier(cleanTe.categories)
# clf.load("clean_sclf.pt")

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 6, 5)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.fc1 = torch.nn.Linear(16 * 34 * 34, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        # relu preserves dimensions
        x = self.pool(torch.nn.functional.relu(self.conv1(x))) # in R^[N, Cout, |X|-|W|+1, |X|-|W|+1]
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        x = x.view(-1, np.prod(x.shape[1:]))
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x
preprocessing = False
clfname = "Resnet50"
ppstr = "PP" if preprocessing else "NoPP"
clffname = f"{clfname}_{ppstr}.pt"
clf = SimpleClassifier(cleanTe.categories, network=torch.load(clffname))

In [5]:
noises = {
#     "dropred": noise.ChannelDropoutNoise(0),
#     "dropgreen": noise.ChannelDropoutNoise(1),
#     "dropblue": noise.ChannelDropoutNoise(2),
    "awn0": noise.AdditiveWhiteNoise(0, 0.1),
    "awn1": noise.AdditiveWhiteNoise(0, 0.2),
    "awn2": noise.AdditiveWhiteNoise(0, 0.3),
    "awn3": noise.AdditiveWhiteNoise(0, 0.4),
    "awn4": noise.AdditiveWhiteNoise(0, 0.5),
    "awn5": noise.AdditiveWhiteNoise(0, 0.6),
    "srn0": noise.ScalarRayleighNoise(0.8*np.sqrt(2.0 / np.pi)),
    "srn1": noise.ScalarRayleighNoise(0.9*np.sqrt(2.0 / np.pi)),
    "srn2": noise.ScalarRayleighNoise(1.0*np.sqrt(2.0 / np.pi)),
    "srn3": noise.ScalarRayleighNoise(1.1*np.sqrt(2.0 / np.pi)),
    "srn4": noise.ScalarRayleighNoise(1.2*np.sqrt(2.0 / np.pi)),
    "drop0": noise.DropoutNoise(0.05, axis=1),
    "drop1": noise.DropoutNoise(0.1, axis=1),
    "drop2": noise.DropoutNoise(0.15, axis=1),
    "drop3": noise.DropoutNoise(0.2, axis=1),
    "drop4": noise.DropoutNoise(0.25, axis=1),
    "drop5": noise.DropoutNoise(0.3, axis=1)
}

recalculate = True

noisefname = {k: os.path.join(baseoutdir, f"Noise_{clfname}_{str(k)}.npz") for k in noises.keys()}


In [6]:
transform = tv.transforms.Compose(
    [tv.transforms.CenterCrop(150),
     tv.transforms.ToTensor(),
     tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batchsize = 8

In [7]:
dsTe = {"clean": cleanTe}
dsTe.update({
    k: isds.IntelNoiseSceneDataset(cleanTe, noises[k], noisefname=(noisefname[k] if not recalculate else None))
        for k in tqdm(noises.keys())
})

In [12]:
if recalculate:
    for k,fname in tqdm(noisefname.items()):
        try:
            np.savez(fname, noises[k])
        except Exception as e:
            print(e.message, e.args)

In [13]:
batchdims = denoise.CSC.batchdims(cleanTe.shape[2:], cleanTe.shape[1], batchsize)
batchkeys = {"sigshape", "channels", "batchsize"}
dictfname = "test_dict.npy"
csc = denoise.CSC(dictfname, batchdims, dimN=2, dtype=torch.float32)

{'C': 3,
 'Cd': 3,
 'K': 8,
 'M': 240,
 'N': 22500,
 'Nv': (150, 150),
 'axisC': 2,
 'axisK': 3,
 'axisM': 4,
 'axisN': (0, 1),
 'dimC': 1,
 'dimCd': 1,
 'dimK': 1,
 'dimN': 2,
 'shpD': (10, 10, 3, 1, 240),
 'shpS': (150, 150, 3, 8, 1),
 'shpX': (150, 150, 1, 8, 240)}


In [14]:
# Allocate Confusion matrices
preprocessor = {"Baseline": None, "CDL": csc.normsolve}
Conf = {
    j: {
        k: np.zeros((len(cleanTe.categories), len(cleanTe.categories)), dtype=np.uint)
            for k in dsTe.keys()
    } for j in preprocessor.keys()
}
SNR = {j: {k: None for k in dsTe.keys()} for j in preprocessor.keys()}
snrFlag = {k: True for k in dsTe.keys()}
snrFlag["clean"] = False

In [15]:
dataloader = torch.utils.data.DataLoader(dsTe["srn0"],
                                             batch_size=batchsize,
                                             shuffle=True,
                                             pin_memory=True,
                                             num_workers=0)

# noise_key = "drop0"
# img_clean = dsTe["clean"].__getitem__(0)["images"].to(device)
# print(img_clean.shape)
# print(dsTe[noise_key].__getitem__(0))
# print(dsTe[noise_key].__getitem__(0).keys())
# img_noise = dsTe[noise_key].__getitem__(0)["images"].to(device)
# img_recon = torch.clamp(csc.solve(img_noise.reshape((1,) + img_noise.shape), normed=True, lmbda=1.0e-2)[0,...], -1, 1).squeeze()

# noise_diff = torch.abs(img_clean - img_noise)
# recon_diff = torch.abs(img_clean - img_recon)

# print(f"(min,max) = ({torch.min(img_clean):.3f}, {torch.max(img_clean):.3f})")
# print(f"(min,max) = ({torch.min(img_noise):.3f}, {torch.max(img_noise):.3f})")
# print(f"(min,max) = ({torch.min(img_recon):.3f}, {torch.max(img_recon):.3f})")

# for m in range(imgs.shape[0]):
#     plt.subplot(2,2,1)
#     plt.imshow(0.5*imgs[m,...].permute((1,2,0)).cpu() + 0.5)
#     plt.subplot(2,2,2)
#     plt.imshow(dimgs[m,...].permute((1,2,0)).cpu())
#     plt.subplot(2,2,3)
#     plt.imshow(0.5*rimgs[m,...].permute((1,2,0)).cpu() + 0.5)
#     plt.subplot(2,2,4)
#     plt.imshow(0.5*torch.abs(rimgs[m,...].permute((1,2,0)).cpu() + 0.5))
#     plt.show()

In [16]:
# cpuperm = lambda x: x.permute(1,2,0).cpu()
# norm = lambda x: 0.5*cpuperm(x) + 0.5
# diffnorm = lambda x: cpuperm(x.div_(torch.max(x)))
# fig = plt.subplots(2,2, figsize=(10,10))
# plt.subplot(2,2,1)
# plt.imshow(norm(img_noise))
# plt.subplot(2,2,2)
# plt.imshow(diffnorm(noise_diff))
# plt.subplot(2,2,3)
# plt.imshow(norm(img_recon))
# plt.subplot(2,2,4)
# plt.imshow(diffnorm(recon_diff))

In [18]:
csc.lmbda = 1.0e-2 # 5.0e-2
for k, data in tqdm(dsTe.items()):
    dataloader = torch.utils.data.DataLoader(data,
                                             batch_size=batchsize,
                                             shuffle=True,
                                             pin_memory=True,
                                             num_workers=0)
    for j in  tqdm(Conf.keys(), leave=False):
        if snrFlag[k]:
            Conf[j][k], SNR[j][k] = clf.predict(dataloader,
                                                calcsnr=snrFlag[k],
                                                window=10,
                                                preprocessor=preprocessor[j])
            print(f"{j} Metrics:")
            print("%s Accuracy: %.4f" % (k, np.sum(np.diag(Conf[j][k])) / np.sum(Conf[j][k])))
            print("%s Mean SNR: %f dB" % (k, np.mean(SNR[j][k])))
        else:
            Conf[j][k], _ = clf.predict(dataloader,
                                        calcsnr=snrFlag[k],
                                        window=10,
                                        preprocessor=preprocessor[j])
            print(f"{j} Metrics:")
            print("%s Accuracy: %.4f" % (k, np.sum(np.diag(Conf[j][k])) / np.sum(Conf[j][k])))
            print("============================================================================")

Baseline Metrics:
clean Accuracy: 0.8527


KeyboardInterrupt: 

In [ ]:
for j,Cdct in Conf.keys():
    for k,C in Cdct.items():
        fname = str(k) + "_" + confcdlfname
        fname = f"Confusion_{clfname}_{str(j)}_{str(k)}.npz"
        pname = os.path.join(baseoutdir, fname)
        try:
            np.savez_compressed(pname, C=C)
        except Exception as e:
            print(e.message, e.args)